In [4]:
import torch
import torchvision
import torch.utils.data.dataloader as Data

train_data = torchvision.datasets.MNIST(
    './mnist', train=True, transform=torchvision.transforms.ToTensor(), download=True
)
test_data = torchvision.datasets.MNIST(
    './mnist', train=False, transform=torchvision.transforms.ToTensor()
)
print("train_data:", train_data.train_data.size())
print("train_labels:", train_data.train_labels.size())
print("test_data:", test_data.test_data.size())


#超参数
batch_size = 64
Epoch = 100
LR = 0.001  # 0.1, 0.001, 0.000001
Weight_decay = 0.005

#数据读取
train_loader = Data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = Data.DataLoader(dataset=test_data, batch_size=batch_size)

#定义模型
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear_0 = torch.nn.Linear(784, 500)
        self.linear_1 = torch.nn.Linear(500, 128)
        self.linear_2 = torch.nn.Linear(128, 50)
        self.linear_3 = torch.nn.Linear(50, 10)
        self.act = torch.nn.ReLU()

    def forward(self, x):
        x = self.linear_0(x)
        x = self.act(x)
        x = self.linear_1(x)
        x = self.act(x)
        x = self.linear_2(x)
        x = self.act(x)
        out = self.linear_3(x)
        return out

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
#初始化模型，优化器
model = Net()
print(model)
print(f'The model has {count_parameters(model):,} trainable parameters')
optimizer = torch.optim.Adam(model.parameters(),lr=LR,weight_decay=Weight_decay)
loss_func = torch.nn.CrossEntropyLoss()


#训练，测试
for epoch in range(Epoch):
    print('epoch {}'.format(epoch + 1))
    # training-----------------------------
    train_loss = 0.
    train_acc = 0.
    cur = 1
    for batch_x, batch_y in train_loader:
        if cur == 1 and epoch == 0:
            print(batch_x.size())
            print(batch_y.size())
            cur += 1
        batch_x = batch_x.view(batch_x.size()[0],-1)
        out = model(batch_x)
        loss = loss_func(out, batch_y)
        train_loss += loss.item()
        pred = torch.max(out, 1)[1]
        train_correct = (pred == batch_y).sum()
        train_acc += train_correct.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Train Loss: {:.6f}, Acc: {:.6f}'.format(train_loss / (len(
        train_data)), train_acc / (len(train_data))))

    # evaluation--------------------------------
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.view(batch_x.size()[0],-1)
        out = model(batch_x)
        loss = loss_func(out, batch_y)
        eval_loss += loss.item()
        pred = torch.max(out, 1)[1]
        num_correct = (pred == batch_y).sum()
        eval_acc += num_correct.item()
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_data)), eval_acc / (len(test_data))))

train_data: torch.Size([60000, 28, 28])
train_labels: torch.Size([60000])
test_data: torch.Size([10000, 28, 28])
Net(
  (linear_0): Linear(in_features=784, out_features=500, bias=True)
  (linear_1): Linear(in_features=500, out_features=128, bias=True)
  (linear_2): Linear(in_features=128, out_features=50, bias=True)
  (linear_3): Linear(in_features=50, out_features=10, bias=True)
  (act): ReLU()
)
The model has 463,588 trainable parameters
epoch 1
torch.Size([64, 1, 28, 28])
torch.Size([64])
Train Loss: 0.006123, Acc: 0.884200
Test Loss: 0.003717, Acc: 0.932300
epoch 2
Train Loss: 0.003287, Acc: 0.940317
Test Loss: 0.002973, Acc: 0.943400
epoch 3
Train Loss: 0.002785, Acc: 0.949967
Test Loss: 0.002301, Acc: 0.957600
epoch 4
Train Loss: 0.002494, Acc: 0.954850
Test Loss: 0.002483, Acc: 0.953700
epoch 5
Train Loss: 0.002362, Acc: 0.958350
Test Loss: 0.002387, Acc: 0.956400
epoch 6
Train Loss: 0.002238, Acc: 0.960017
Test Loss: 0.001929, Acc: 0.966400
epoch 7
Train Loss: 0.002182, Acc: 0.

Test Loss: 0.001864, Acc: 0.964300
epoch 98
Train Loss: 0.001916, Acc: 0.966883
Test Loss: 0.001864, Acc: 0.965800
epoch 99
Train Loss: 0.001904, Acc: 0.967483
Test Loss: 0.002033, Acc: 0.964600
epoch 100
Train Loss: 0.001910, Acc: 0.966667
Test Loss: 0.001805, Acc: 0.970000
